In [17]:
import pandas as pd
import numpy as np

In [2]:
df_drug = pd.read_csv('../data/DrugCount.csv')

In [3]:
df_drug.head()

,MemberID,Year,DSFS,DrugCount
0,48925661,Y2,9-10 months,7+
1,90764620,Y3,8- 9 months,3
2,61221204,Y1,2- 3 months,1
3,63628544,Y3,1- 2 months,1
4,46949606,Y2,10-11 months,3


In [5]:
def count_nulls(df):
    null_counter = df.isnull().sum(axis=0)
    #null_counter = null_counter[null_counter > 0]
    null_percent = df.isnull().sum(axis=0) / df.shape[0] * 100
    #null_percent = null_percent[null_percent > 0]
    null_df = pd.concat([null_counter,null_percent],axis=1)
    null_df.columns = ['null_count','null_percent']
    display(null_df)
    
count_nulls(df_drug)

,null_count,null_percent
MemberID,0,0.0
Year,0,0.0
DSFS,0,0.0
DrugCount,0,0.0


In [10]:
# DSFS 
   # Convert from categorical to numerical type
def updateDSFS():
    df_drug.loc[df_drug['DSFS']=='0- 1 month','DSFS'] = 15
    df_drug.loc[df_drug['DSFS']=='1- 2 months','DSFS'] = 45
    df_drug.loc[df_drug['DSFS']=='2- 3 months','DSFS'] = 75
    df_drug.loc[df_drug['DSFS']=='3- 4 months','DSFS'] = 105
    df_drug.loc[df_drug['DSFS']=='4- 5 months','DSFS'] = 135
    df_drug.loc[df_drug['DSFS']=='5- 6 months','DSFS'] = 165
    df_drug.loc[df_drug['DSFS']=='6- 7 months','DSFS'] = 195
    df_drug.loc[df_drug['DSFS']=='7- 8 months','DSFS'] = 225
    df_drug.loc[df_drug['DSFS']=='8- 9 months','DSFS'] = 255
    df_drug.loc[df_drug['DSFS']=='9-10 months','DSFS'] = 285
    df_drug.loc[df_drug['DSFS']=='10-11 months','DSFS'] = 315
    df_drug.loc[df_drug['DSFS']=='11-12 months','DSFS'] = 345
    df_drug['DSFS'].fillna(0, inplace=True)
    df_drug['DSFS'] = df_drug['DSFS'].astype(int)
    return

updateDSFS()

In [14]:
# DrugCount
   # Convert from text to numerical type
def updateDrugCount():
    df_drug.loc[df_drug['DrugCount']=='1','DrugCount'] = 1
    df_drug.loc[df_drug['DrugCount']=='2','DrugCount'] = 2
    df_drug.loc[df_drug['DrugCount']=='3','DrugCount'] = 3
    df_drug.loc[df_drug['DrugCount']=='4','DrugCount'] = 4
    df_drug.loc[df_drug['DrugCount']=='5','DrugCount'] = 5
    df_drug.loc[df_drug['DrugCount']=='6','DrugCount'] = 6
    df_drug.loc[df_drug['DrugCount']=='7+','DrugCount'] = 7
    df_drug['DrugCount'].fillna(0, inplace=True)
    df_drug['DrugCount'] = df_drug['DrugCount'].astype(int)
    return

updateDrugCount()

In [18]:
# Aggregation on (Year, MemberID)
   #Helper functions
def std(x):
    if len(x) == 1:
        return 0
    else:
        return np.std(x)

df_drug_summary = df_drug.groupby(['Year', 'MemberID']).agg({
    'MemberID': 'count',
    'DrugCount': ['sum', 'max', 'min', 'mean', std]
})

In [20]:
# Change column names
df_drug_summary.columns = [
    'no_drug_instances',
    'DrugCount_sum',
    'DrugCount_max',
    'DrugCount_min',
    'DrugCount_mean',
    'DrugCount_std'
]

# Reset index
df_drug_summary = df_drug_summary.reset_index()

In [22]:
# Export to csv for further processing
df_drug_summary.to_csv("processed-data/drug_summary.csv", index=False)